## Inference on Tiny-Mamba Yolo


In [1]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance
from ultralytics import YOLO
import supervision as sv


## Process "Transform" on video and save it.

In [8]:
def transform(image):
    img = image.resize((1024, 1024), Image.Resampling.LANCZOS)
    sharp = ImageEnhance.Sharpness(img)
    img = sharp.enhance(6) 
    contrast = ImageEnhance.Contrast(img)
    img = contrast.enhance(6)  
    #img = img.resize((1024, 1024)) 
    return img

input_video_path = 'videos/video.mp4'
output_video_path = 'videos/video_transformed.mp4'

cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print("Error: Couldn't open video file.")
    exit()

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 'mp4v' codec for mp4 files
out = cv2.VideoWriter(output_video_path, fourcc, fps, (1024, 1024))  # Output size: 1024x1024

while True:
    ret, frame = cap.read()
    if not ret:
        break  
    frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    transformed_frame = transform(frame_pil)
    transformed_frame_cv = cv2.cvtColor(np.array(transformed_frame), cv2.COLOR_RGB2BGR)
    out.write(transformed_frame_cv)

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video saved as {output_video_path}")


Video saved as videos/video_transformed.mp4


## Inference on transformed video

In [ ]:
# Load YOLO model (using the trained model file)
MODEL_PATH = "models/last.pt"
model = YOLO()
print(model)
video_info = sv.VideoInfo.from_video_path(output_video_path)
results = model.predict(output_video_path, stream=True,imgsz=(1024,1024))
for result in results:
    boxes = result.boxes  
    masks = result.masks 
    keypoints = result.keypoints 
    probs = result.probs 
    obb = result.obb 

    result.show() 


## Infernece on Raw video (not transformed)

In [ ]:
from ultralytics import YOLO
MODEL_PATH = "models/last.pt"
model = YOLO(MODEL_PATH)
print(model)
# Raw Plume youtube video.
source = "https://www.youtube.com/watch?v=nR_9YSX7T9I"

results = model.predict(source,stream=True,imgsz=(1024,1024)) 
for result in results:
    boxes = result.boxes  
    masks = result.masks 
    keypoints = result.keypoints 
    probs = result.probs 
    obb = result.obb 

    result.show() 